In [ ]:
%matplotlib inline
!pip install wandb
!wandb login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
wandb: Currently logged in as: omersiton. Use `wandb login --relogin` to force relogin


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset
from torch.utils.data import dataloader
import torch.nn as nn
import torch.nn.functional as F
import wandb
import torch.optim as optim

## Load and normalize CIFAR10

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 5

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainset_subset = Subset(trainset, range(50000))

trainloader = torch.utils.data.DataLoader(trainset_subset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
print(len(trainloader))

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

testset_subset = Subset(testset, range(10000))

testloader = torch.utils.data.DataLoader(testset_subset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
print(len(testloader))

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
10000
Files already downloaded and verified
2000


##Helper Functions

###Initalize W&B

In [ ]:
def init_wb():
  # start a new wandb run to track this script
  wandb.init(
      # set the wandb project where this run will be logged
      project="NN.Ex1",
      
      # track hyperparameters and run metadata
      config={
      "learning_rate": 0.001,
      "architecture": "CNN",
      "dataset": "CIFAR-10",
      "epochs": 10,
      "device": device,
      "#neurons": sum(p.numel() for p in net.parameters() if p.requires_grad)
      }
  )


### Accuracy 

In [ ]:
def accuracy():
  correct = 0
  total = 0
  # since we're not training, we don't need to calculate the gradients for our outputs
  with torch.no_grad():
      for data in testloader:
          images, labels = data[0].to(device), data[1].to(device)
          # calculate outputs by running images through the network
          outputs = net(images)
          # the class with the highest energy is what we choose as prediction
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

### Accuracy by Class


In [ ]:
def accuracy_by_class():
  # prepare to count predictions for each class
  correct_pred = {classname: 0 for classname in classes}
  total_pred = {classname: 0 for classname in classes}

  # again no gradients needed
  with torch.no_grad():
      for data in testloader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs = net(images)
          _, predictions = torch.max(outputs, 1)
          # collect the correct predictions for each class
          for label, prediction in zip(labels, predictions):
              if label == prediction:
                  correct_pred[classes[label]] += 1
              total_pred[classes[label]] += 1


  # print accuracy for each class
  for classname, correct_count in correct_pred.items():
      accuracy = 100 * float(correct_count) / total_pred[classname]
      print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

### Train the network


In [ ]:

def train_net():
  for epoch in range(wandb.config.epochs):  # loop over the dataset multiple times

      running_loss = 0.0
      train_loss = 0.0
      for i, data in enumerate(trainloader, 0):
          # get the images; data is a list of [images, labels]
          images, labels = data[0].to(device), data[1].to(device)

          # zero the parameter gradients
          optimizer.zero_grad()

          # forward + backward + optimize
          outputs = net(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          # print statistics
          running_loss += loss.item()
          if i % 2000 == 1999:    # print every 2000 mini-batches
              print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
              train_loss += running_loss / 2000
              running_loss = 0.0

      # compute and print test loss at the end of each epoch
      test_loss = 0.0
      with torch.no_grad():
          for data in testloader:
              images, labels = data[0].to(device), data[1].to(device)
              outputs = net(images)
              test_loss += criterion(outputs, labels).item()

      print('Epoch %d, Test loss: %.3f' % (epoch + 1, test_loss / len(testloader)))
      wandb.log({'Test loss / Epoch' : test_loss / len(testloader),
                'Train loss / Epoch' : train_loss / int(len(trainloader)/2000)})

  print('Finished Training')

## Define Convolutional Neural Network Classes

In [ ]:

class Block(nn.Module):
    def __init__(self, in_dim, out_dim, depth=1):
      super().__init__()
      self.depth = depth
      self.layers = nn.ModuleList([])
      self.layers.append(nn.Conv2d(in_dim, out_dim, 3, padding='same'))
      for i in range(1,depth):
        self.layers.append(nn.Conv2d(out_dim, out_dim, 3, padding='same'))
      self.pool = nn.MaxPool2d(2, 2)


    def forward(self, x):
      skip_connection = F.relu(self.layers[0](x))
      for i, layer in enumerate(self.layers):
          x = F.relu(layer(x) + (skip_connection if i % 2 == 0 and self.depth > 2 else 0))
      x = self.pool(x)
      return x


class Net(nn.Module):
    def __init__(self, net_depth, block_depth, in_channels=3, out_channels=10):
        super().__init__()
        self.layers = nn.ModuleList([])
        inner_dim = 16
        HW = 32
        for i in range(net_depth):
          self.layers.append(Block(in_channels, inner_dim, depth=block_depth))
          in_channels = inner_dim
          inner_dim = inner_dim * 4 if i != net_depth-1 else inner_dim
          HW /= 2
        
        self.fc1 = nn.Linear(inner_dim * int(HW) * int(HW), out_channels)
        

    def forward(self, x):
        for layer in self.layers:
          x = layer(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x


for net_depth in [1, 2, 3]:
  for block_depth in [1, 3]:
    # print depths
    print(f"Network depth - {net_depth}, Block depth - {block_depth}")
    net = Net(net_depth, block_depth)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    net.to(device)
    # print the number of learning parameters
    print(sum(p.numel() for p in net.parameters() if p.requires_grad))
    # init logging service W&B
    init_wb()
    # define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=wandb.config.learning_rate, momentum=0.9)
    # train
    train_net()
    # calc accuracy
    accuracy()
    accuracy_by_class()




Network depth - 1, Block depth - 1
41418


Test loss / Epoch,█▅▃▂▁▁▂▆▅▅▅
Train loss / Epoch,█▅▄▃▂▁▁▆▅▄▄
Test loss / Epoch,1.14152
Train loss / Epoch,1.03984


[1,  2000] loss: 1.766
[1,  4000] loss: 1.455
[1,  6000] loss: 1.394
[1,  8000] loss: 1.344
[1, 10000] loss: 1.320
Epoch 1, Test loss: 1.277
[2,  2000] loss: 1.242
[2,  4000] loss: 1.202
[2,  6000] loss: 1.218
[2,  8000] loss: 1.222
[2, 10000] loss: 1.172
Epoch 2, Test loss: 1.176
[3,  2000] loss: 1.121
[3,  4000] loss: 1.114
[3,  6000] loss: 1.117
[3,  8000] loss: 1.117
[3, 10000] loss: 1.106
Epoch 3, Test loss: 1.188
[4,  2000] loss: 1.037
[4,  4000] loss: 1.048
[4,  6000] loss: 1.047
[4,  8000] loss: 1.070
[4, 10000] loss: 1.077
Epoch 4, Test loss: 1.134
[5,  2000] loss: 0.988
[5,  4000] loss: 1.014
[5,  6000] loss: 1.018
[5,  8000] loss: 1.021
[5, 10000] loss: 1.039
Epoch 5, Test loss: 1.142
[6,  2000] loss: 0.940
[6,  4000] loss: 0.972
[6,  6000] loss: 0.981
[6,  8000] loss: 1.007
[6, 10000] loss: 1.015
Epoch 6, Test loss: 1.194
[7,  2000] loss: 0.918
[7,  4000] loss: 0.939
[7,  6000] loss: 0.963
[7,  8000] loss: 0.981
[7, 10000] loss: 0.971
Epoch 7, Test loss: 1.238
[8,  2000] lo

Test loss / Epoch,█▃▄▁▁▄▆▅▆▆
Train loss / Epoch,█▅▄▃▂▂▂▁▁▁
Test loss / Epoch,1.22631
Train loss / Epoch,0.90018


[1,  2000] loss: 1.717
[1,  4000] loss: 1.405
[1,  6000] loss: 1.303
[1,  8000] loss: 1.238
[1, 10000] loss: 1.191
Epoch 1, Test loss: 1.187
[2,  2000] loss: 1.055
[2,  4000] loss: 1.060
[2,  6000] loss: 1.025
[2,  8000] loss: 1.027
[2, 10000] loss: 1.030
Epoch 2, Test loss: 1.086
[3,  2000] loss: 0.910
[3,  4000] loss: 0.938
[3,  6000] loss: 0.943
[3,  8000] loss: 0.938
[3, 10000] loss: 0.949
Epoch 3, Test loss: 1.011
[4,  2000] loss: 0.835
[4,  4000] loss: 0.853
[4,  6000] loss: 0.870
[4,  8000] loss: 0.885
[4, 10000] loss: 0.905
Epoch 4, Test loss: 1.038
[5,  2000] loss: 0.774
[5,  4000] loss: 0.826
[5,  6000] loss: 0.830
[5,  8000] loss: 0.844
[5, 10000] loss: 0.858
Epoch 5, Test loss: 1.068
[6,  2000] loss: 0.742
[6,  4000] loss: 0.782
[6,  6000] loss: 0.788
[6,  8000] loss: 0.821
[6, 10000] loss: 0.830
Epoch 6, Test loss: 1.060
[7,  2000] loss: 0.717
[7,  4000] loss: 0.728
[7,  6000] loss: 0.764
[7,  8000] loss: 0.811
[7, 10000] loss: 0.799
Epoch 7, Test loss: 1.083
[8,  2000] lo

...

[1,  2000] loss: 1.782
[1,  4000] loss: 1.401
[1,  6000] loss: 1.259
[1,  8000] loss: 1.140
[1, 10000] loss: 1.065
Epoch 1, Test loss: 1.054
[2,  2000] loss: 0.966
[2,  4000] loss: 0.935
[2,  6000] loss: 0.919
[2,  8000] loss: 0.885
[2, 10000] loss: 0.857
Epoch 2, Test loss: 0.969
[3,  2000] loss: 0.759
[3,  4000] loss: 0.775
[3,  6000] loss: 0.767
[3,  8000] loss: 0.739
[3, 10000] loss: 0.743
Epoch 3, Test loss: 0.809
[4,  2000] loss: 0.632
[4,  4000] loss: 0.649
[4,  6000] loss: 0.663
[4,  8000] loss: 0.675
[4, 10000] loss: 0.678
Epoch 4, Test loss: 0.796
[5,  2000] loss: 0.557
[5,  4000] loss: 0.576
[5,  6000] loss: 0.608
[5,  8000] loss: 0.608
[5, 10000] loss: 0.608
Epoch 5, Test loss: 0.834
[6,  2000] loss: 0.492
[6,  4000] loss: 0.541
[6,  6000] loss: 0.533
[6,  8000] loss: 0.560
[6, 10000] loss: 0.578
Epoch 6, Test loss: 0.845
[7,  2000] loss: 0.434
[7,  4000] loss: 0.468
[7,  6000] loss: 0.512
[7,  8000] loss: 0.544
[7, 10000] loss: 0.533
Epoch 7, Test loss: 0.918
[8,  2000] lo

Test loss / Epoch,█▆▁▁▂▂▄▃▄▄
Train loss / Epoch,█▅▄▃▂▂▂▁▁▁
Test loss / Epoch,0.91243
Train loss / Epoch,0.42826


[1,  2000] loss: 1.919
[1,  4000] loss: 1.535
[1,  6000] loss: 1.381
[1,  8000] loss: 1.277
[1, 10000] loss: 1.186
Epoch 1, Test loss: 1.172
[2,  2000] loss: 1.094
[2,  4000] loss: 1.018
[2,  6000] loss: 1.004
[2,  8000] loss: 0.944
[2, 10000] loss: 0.923
Epoch 2, Test loss: 0.907
[3,  2000] loss: 0.839
[3,  4000] loss: 0.817
[3,  6000] loss: 0.814
[3,  8000] loss: 0.790
[3, 10000] loss: 0.781
Epoch 3, Test loss: 0.870
[4,  2000] loss: 0.669
[4,  4000] loss: 0.696
[4,  6000] loss: 0.691
[4,  8000] loss: 0.695
[4, 10000] loss: 0.683
Epoch 4, Test loss: 0.811
[5,  2000] loss: 0.559
[5,  4000] loss: 0.585
[5,  6000] loss: 0.613
[5,  8000] loss: 0.590
[5, 10000] loss: 0.603
Epoch 5, Test loss: 0.806
[6,  2000] loss: 0.455
[6,  4000] loss: 0.497
[6,  6000] loss: 0.497
[6,  8000] loss: 0.526
[6, 10000] loss: 0.534
Epoch 6, Test loss: 0.873
[7,  2000] loss: 0.380
[7,  4000] loss: 0.407
[7,  6000] loss: 0.424
[7,  8000] loss: 0.461
[7, 10000] loss: 0.452
Epoch 7, Test loss: 0.925
[8,  2000] lo

Test loss / Epoch,█▃▂▁▁▂▃▃▅▆
Train loss / Epoch,█▅▄▄▃▂▂▂▁▁
Test loss / Epoch,1.09087
Train loss / Epoch,0.25563


[1,  2000] loss: 1.874
[1,  4000] loss: 1.479
[1,  6000] loss: 1.298
[1,  8000] loss: 1.158
[1, 10000] loss: 1.046
Epoch 1, Test loss: 1.037
[2,  2000] loss: 0.918
[2,  4000] loss: 0.878
[2,  6000] loss: 0.848
[2,  8000] loss: 0.814
[2, 10000] loss: 0.777
Epoch 2, Test loss: 0.777
[3,  2000] loss: 0.654
[3,  4000] loss: 0.658
[3,  6000] loss: 0.655
[3,  8000] loss: 0.642
[3, 10000] loss: 0.641
Epoch 3, Test loss: 0.690
[4,  2000] loss: 0.502
[4,  4000] loss: 0.516
[4,  6000] loss: 0.504
[4,  8000] loss: 0.531
[4, 10000] loss: 0.546
Epoch 4, Test loss: 0.672
[5,  2000] loss: 0.371
[5,  4000] loss: 0.391
[5,  6000] loss: 0.418
[5,  8000] loss: 0.429
[5, 10000] loss: 0.458
Epoch 5, Test loss: 0.693
[6,  2000] loss: 0.283
[6,  4000] loss: 0.306
[6,  6000] loss: 0.351
[6,  8000] loss: 0.353
[6, 10000] loss: 0.370
Epoch 6, Test loss: 0.728
[7,  2000] loss: 0.212
[7,  4000] loss: 0.254
[7,  6000] loss: 0.288
[7,  8000] loss: 0.293
[7, 10000] loss: 0.327
Epoch 7, Test loss: 0.783
[8,  2000] lo